In [1]:
# Imports
import multiprocessing
multiprocessing.set_start_method('spawn', True)

import sys
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW
from tqdm import tqdm
import torch.nn as nn
import numpy as np
import torch.cuda.amp as amp
from torch.cuda.amp import autocast, GradScaler


# Verify CUDA
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('Usando GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('Usando CPU')
    

print(sys.version)


Usando GPU: NVIDIA GeForce RTX 4080
3.11.5 (tags/v3.11.5:cce6ba9, Aug 24 2023, 14:38:34) [MSC v.1936 64 bit (AMD64)]


In [2]:

import pandas as pd

# Load the CSV files into DataFrames
df = pd.read_csv('opiniones.csv', encoding='utf-8')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39256 entries, 0 to 39255
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   data    39256 non-null  object
 1   target  39256 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 613.5+ KB


In [3]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import numpy as np 


# Load and prepare data
MODEL_NAME = "dccuchile/bert-base-spanish-wwm-uncased"
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)
model.to(device)

# Check if cuda is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')


# Define dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        text = row['data']
        labels = row['target']
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        return {
            'ids': torch.tensor(inputs['input_ids'], dtype=torch.long),
            'mask': torch.tensor(inputs['attention_mask'], dtype=torch.long),
            'targets': torch.tensor(labels, dtype=torch.long)
        }

# optimizer, sheduler and scaler config
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(df)*3)

scaler = torch.cuda.amp.GradScaler()

# model training function
def train_epoch(model, data_loader, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0

    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)

        with torch.cuda.amp.autocast():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=targets)
            loss = outputs.loss
            logits = outputs.logits

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scheduler.step()

        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == targets)
        losses.append(loss.item())

    return correct_predictions.double() / n_examples, np.mean(losses)

# Evaluar FUNC

def eval_model(model, data_loader, device, n_examples):
    model = model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=targets)
            loss = outputs.loss
            logits = outputs.logits

            _, preds = torch.max(logits, dim=1)
            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())

    return correct_predictions.double() / n_examples, np.mean(losses)

# Dynamically adapting tokenizer length
max_length_from_data = max([len(tokenizer.encode(text)) for text in df['data']])
MAX_LEN = min(max_length_from_data, 512)  # no more than 512 tokens

print(f"Max Length: {MAX_LEN}")

# Constants
EPOCHS = 1  # Define the number of epochs you want to train
BATCH_SIZE = 54  # Adjust based on your system’s capabilities

# Assuming dataset is an instance of your CustomDataset
dataset = CustomDataset(df, tokenizer, MAX_LEN)

# Splitting the dataset into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

scaler = GradScaler()

for epoch in range(EPOCHS):
    model.train()
    
    train_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for i, batch in tqdm(enumerate(train_loader), total=len(train_loader)):
        ids = batch['ids'].to(device)
        mask = batch['mask'].to(device)
        targets = batch['targets'].to(device)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():  # Enable mixed precision
            outputs = model(ids, attention_mask=mask, labels=targets)
            loss = outputs.loss
            logits = outputs.logits

        # Backward and optimize with scaler
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        scheduler.step()  # Update learning rate

        train_loss += loss.item()
        
        _, preds = torch.max(logits, dim=1)
        correct_train += (preds == targets).sum().item()
        total_train += targets.size(0)

    avg_train_loss = train_loss / len(train_loader)
    train_accuracy = correct_train / total_train
    
    model.eval()
    
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for i, batch in tqdm(enumerate(val_loader), total=len(val_loader)):
            ids = batch['ids'].to(device)
            mask = batch['mask'].to(device)
            targets = batch['targets'].to(device)

            with autocast():
                outputs = model(ids, attention_mask=mask, labels=targets)
                loss = outputs.loss
                logits = outputs.logits

            val_loss += loss.item()
            
            _, preds = torch.max(logits, dim=1)
            correct_val += (preds == targets).sum().item()
            total_val += targets.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = correct_val / total_val
    
    print(f"Epoch: {epoch+1}/{EPOCHS}, Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.weight', 'classifier.weight', 'bert.pooler.dense.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using device: cuda
Max Length: 313


100%|██████████████████████████████████████████████████████████████████████████████████| 73/73 [00:10<00:00,  7.11it/s]

Epoch: 1/1, Training Loss: 0.1102, Training Accuracy: 0.9615, Validation Loss: 0.0434, Validation Accuracy: 0.9845


In [4]:
test_texts = [  
    "Me hecho 3 mamografías 2 ecosmamarias porq me sale un nódulo de 10cm pero después de la operación de vesícula en solo semanas creció 5 centímetros y ahora solo esperar q me llamen de salud pública por no tener dinero tengo q esperar q me llame el especialista para la biopsia 😔",
    "Recien a los 45 me empiezan hacer la mamografia preventiva en el cesfam... Como puedo optar a ella antes??",
    "Programas disponibles? Donde? Si es casi imposible llegar a salud mental a través de la salud pública, a no ser que uno intente suicidarse 🤷🏾‍♀️ La concientización no sirve si no existen los medios.",
    "Excelente iniciativa 🙌🏽❤️ Visibilizar y hacernos responsables como sociedad es fundamental",
    "Por eso aumenten la contratación de matronas en Cefam!!! En @cesfamreinaisabeliioficial llevo meses esperando que me vean, me imagino como debe ser para las mujeres mayores,peor todavia!! @cmvalparaiso",
    "No me parece que se ocupen dineros públicos en tratamientos que no tienen evidencia científica",
    "Ese Director del CESFAM atendía a mi hijo en el Mena. Qué lindo que alguien como él este en ese cargo 🙌",
    "Que buena noticia, estos enfoques visualiza el todo y no las partes y posibilita un auto análisis para ser responsable de la promoción y prevención de la salud. Espero se puedan abrir más espacios así, siempre existieran personas que les sea de mucha utilidad ❤️",
    "Medicina alternativa! Excelente 👏👏👏",
    "Y los que tienen horas agendada?",
    "Peguense la cacha y don respuesta a los miles de porteros que hoy están sufriendo de trombosis, acv, paros cardiaco, turbocancer por sus famosas vacunas...Los datos los pueden encontrar en deis.minsal.cl",
    "Felicitaciones por su gran labor y el cariño que siempre ponen !!!",
    "Recordar siempre que el consentimiento se puede quitar en cualquier instante de una relación sexual 🙏🏽",
    "Los funcionarios tambien necesitamos atencion en salud mental.. para entregar un servicio de calidad tambien debemos estar bien fisica y psicologicamente..",
    "Trabajo en educación y a diario se ve lo saturado de los centros de salud familiar, tienen lista de espera enormes y niños y niñas sin atención ! Nadie se hace cargo",
    "Que lindo los videos, pero feo es ver a adultos mayores estar desde las 8:00hrs a las 17:00hrs esperando por renovar su licencia",
    "La directora del cesfam Rodelillo, podría pedir que mejoren la atención. Horrible cómo atienden a las personas.",
    "Cual de todos maneja más mal el Cesfam de los sectores señalados… Placeres vale callampa desde hace por los menos 20 años…",
    "el de esperanza es de viña?",
    "Planilla...tuve una urgencia a las 22:30 hasta y el DOCTOR DE TURNO se había ido a las 22 hrs y llegaba a las 00 hrs...en resumen los pacientes quedan sin atención de urgencia...preocuparse 😮",
    "Como ciudadana les indico que este último tiempo, he vivido una serie de negligencias por parte del servicio de salud de Valparaíso.",
    "Placeres Salud? Porfavor fui una vez que cosa más insaluble ni papel para el baño ,una suciedad",
    "Hacen el ridiculo al no conocer para lo que fueron designado Rina Isabel",
    "Chile cada día peor. De las estupideces que andan preocupados",
    "Que le vaya excelente y que logre solucionar la problemática de farmacia.. la lentitud es increíble... abuelos esperando 2 horas para recibir sus medicamentos es realmente injusto",
    "Ojalá que con esto mejore la gestión de las horas médicas y la disponibilidad de los profesionales.",
    "Me hubiese gustado participar, y porder haber compartido todos mis conocimientos, para la proxima sera ❤️",
    "Increíble labor la que se hace en APS allá, a seguir fortaleciendo esas estrategias y crear otras!!🙌❤️",
    "Felicidades, los recuerdo con mucho cariños. Aprendí demasiado como interno y despues como profesional, gracias por abrirme.las puertas en mi primera experiencia de trabajo en la salud❤️❤️❤️🔥🔥🔥",
    "Deberían ver el tema de anticonceptivos 😴",
    "Buenas me mandan el link de inscripción porfi?",
    "Hola, yo quiero saber para cuándo va a estar disponible la vacuna contra la influenza para todo público. Creo que hasta el momento es para personas de riesgo de la tercera edad y embarazadas. Saludos. 💉",
    
    "El personal del centro de salud siempre ha sido muy atento y profesional. Estoy agradecido por la atención recibida.",
    "Es una lástima que los equipos médicos estén tan desactualizados, afecta la calidad del diagnóstico y tratamiento.",
    "No hay suficiente personal para atender a todos los pacientes, las esperas son eternas.",
    "Increíble el compromiso de los trabajadores de este centro, a pesar de las adversidades siempre dan lo mejor.",
    "Deberían mejorar la comunicación entre los departamentos para evitar errores y malentendidos.",
    "Me cancelaron la cita el mismo día, no pueden ser tan desorganizados.",
    "Los baños están sucios y no hay suficientes medidas de higiene en las instalaciones.",
    "Excelente la atención del pediatra, muy profesional y cariñoso con los niños.",
    "Las instalaciones son antiguas y pequeñas, no están a la altura de las necesidades actuales.",
    "A veces siento que los médicos no tienen suficiente tiempo para atender y escuchar a los pacientes adecuadamente.",
    "Sería bueno tener más especialistas, para no tener que esperar meses por una cita.",
    "Los administrativos son muy groseros y hacen que la experiencia en el centro sea desagradable.",
    "Las enfermeras hacen un gran trabajo, siempre están dispuestas a ayudar.",
    "¿Podrían mejorar la señalización dentro del centro? Es fácil perderse.",
    "Hay mucha demora para la entrega de medicamentos, deberían ser más eficientes.",
    "Gracias por ofrecer charlas y talleres para la comunidad, son de mucha ayuda.",
    "La página web siempre está desactualizada, no se puede confiar en la información que aparece.",
    "La atención es buena, pero los costos de los tratamientos son muy altos.",
    "Esperé más de una hora más allá de mi cita, deberían respetar más el tiempo de los pacientes.",
    "Los servicios en línea son muy prácticos y facilitan muchos trámites. Buen trabajo en eso.",
    
    "Aunque las instalaciones son algo antiguas, el personal hace un trabajo extraordinario manteniendo todo limpio y organizado.",
    "Los trámites administrativos son un dolor de cabeza; sin embargo, los médicos y enfermeras son realmente compasivos y profesionales.",
    "La espera fue larga y el lugar estaba algo desordenado, pero el médico fue muy atento y resolvió todas mis dudas.",
    "Los procesos online podrían ser más intuitivos. Me costó mucho realizar mi agendamiento, pero el seguimiento fue constante y preciso.",
    "Aprecio la variedad de especialistas disponibles, pero mejorar la coordinación entre los diferentes departamentos haría la experiencia mucho mejor.",
    "Increíble la dedicación del personal, pero las instalaciones necesitan una renovación urgente para estar a la altura.",
    "Los servicios de urgencias son eficientes, pero las áreas de espera podrían ser más cómodas y acogedoras.",
    "La atención telefónica es impecable, pero la página web necesita mejoras para facilitar la navegación.",
    "El centro ofrece una amplia gama de servicios, pero encontrar estacionamiento es una verdadera odisea.",
    "El equipo médico es altamente calificado, pero a veces la comunicación con el paciente podría ser más clara y empática.",
    "La rapidez en la atención de emergencias es destacable, pero el seguimiento post-visita podría ser más consistente.",
    "Los servicios preventivos y los programas de bienestar son excelentes, pero la atención en casos de emergencia podría ser más rápida.",
    "La calidad de la atención es muy buena, pero sería ideal que implementaran un sistema de recordatorios para las citas y medicamentos."
]

test_targets = [  
    1, 0, 1, 2, 1, 0, 2, 2, 2, 0, 1, 2, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 2, 2, 0, 0, 0,
    2,  # Positivo
    1,  # Negativo
    1,  # Negativo
    2,  # Positivo
    1,  # Negativo
    1,  # Negativo
    1,  # Negativo
    2,  # Positivo
    1,  # Negativo
    1,  # Negativo
    1,  # Negativo
    1,  # Negativo
    2,  # Positivo
    1,  # Negativo
    1,  # Negativo
    2,  # Positivo
    1,  # Negativo
    1,  # Negativo
    1,  # Negativo
    2,  # Positivo
    
    2,  # Positivo
    1,  # Negativo
    1,  # Negativo
    1,  # Negativo
    1,  # Negativo
    1,  # Negativo
    1,  # Negativo
    1,  # Negativo
    1,  # Negativo
    1,  # Negativo
    1,  # Negativo
    1,  # Negativo
    1   # Negativo
]

In [5]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, BertForSequenceClassification

def predict_sentiments(texts, targets, model, tokenizer, max_len):
    """Predict the sentiment of a list of texts using a provided model and tokenizer."""

    label_map = {
        0: "Irrelevante",
        1: "Negativo",
        2: "Positivo",
    }

    encodings = tokenizer.batch_encode_plus(
        texts,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors="pt",
    )

    input_ids = encodings["input_ids"].to(device)
    attention_mask = encodings["attention_mask"].to(device)

    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        probs = F.softmax(outputs.logits, dim=1)
        _, preds = torch.max(outputs.logits, dim=1)  # Get the predicted classes

    correct_predictions = (preds.cpu() == torch.Tensor(targets)).sum().item()
    accuracy = (correct_predictions / len(targets)) * 100

    warning_count = 0
    correct_count = 0
    incorrect_count = 0
    results = []
    
    for text, target, pred, prob in zip(texts, targets, preds, probs):
        predicted_label = label_map[pred.item()]
        confidence = prob[pred].item() * 100  # Convert to percentage
        true_label = label_map[target]
        
        correctness_emoji = ""
        if pred.item() == target:
            correctness_emoji = "✅"
            correct_count += 1
        else:
            correctness_emoji = "❌"
            incorrect_count += 1
        
        warning_emoji = ""
        if confidence < 90:
            warning_emoji = "⚠️"
            warning_count += 1

        results.append(f"'{text}'\nFue predecido como: '{predicted_label}' con: '{confidence:.2f}% de confianza'.\nLa etiqueta correcta es: '{true_label}'.\nInfo: {correctness_emoji} {warning_emoji}\n")

    print(f"Precisión general: {accuracy:.2f}%\n")
    print(f"Correctos: {correct_count} ✅\nIncorrectos: {incorrect_count} ❌\nAdvertencias: {warning_count} ⚠️\n")
    print("\n".join(results))
    
    return preds, probs

# Assuming you have your model and tokenizer already loaded, and texts and targets defined
MAX_LEN = max([len(tokenizer.encode(text)) for text in test_texts])  # Or any number based on your needs

# Predict sentiments for the list of texts
sentiments, probabilities = predict_sentiments(test_texts, test_targets, model, tokenizer, MAX_LEN)


Precisión general: 100.00%

Correctos: 65 ✅
Incorrectos: 0 ❌
Advertencias: 1 ⚠️

'Me hecho 3 mamografías 2 ecosmamarias porq me sale un nódulo de 10cm pero después de la operación de vesícula en solo semanas creció 5 centímetros y ahora solo esperar q me llamen de salud pública por no tener dinero tengo q esperar q me llame el especialista para la biopsia 😔'
Fue predecido como: 'Negativo' con: '99.74% de confianza'.
La etiqueta correcta es: 'Negativo'.
Info: ✅ 

'Recien a los 45 me empiezan hacer la mamografia preventiva en el cesfam... Como puedo optar a ella antes??'
Fue predecido como: 'Irrelevante' con: '99.83% de confianza'.
La etiqueta correcta es: 'Irrelevante'.
Info: ✅ 

'Programas disponibles? Donde? Si es casi imposible llegar a salud mental a través de la salud pública, a no ser que uno intente suicidarse 🤷🏾‍♀️ La concientización no sirve si no existen los medios.'
Fue predecido como: 'Negativo' con: '99.83% de confianza'.
La etiqueta correcta es: 'Negativo'.
Info: ✅ 

'Exce

In [6]:
# Save the model weights and configuration
model.save_pretrained('./model_directory')

# Save the tokenizer
tokenizer.save_pretrained('./model_directory')

('./model_directory\\tokenizer_config.json',
 './model_directory\\special_tokens_map.json',
 './model_directory\\vocab.txt',
 './model_directory\\added_tokens.json')